# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 12:13PM,253834,20,8955467,"Exact-Rx, Inc.",Released
1,Dec 27 2022 12:13PM,253834,20,8955468,"Exact-Rx, Inc.",Released
2,Dec 27 2022 12:13PM,253834,20,8955469,"Exact-Rx, Inc.",Released
3,Dec 27 2022 12:13PM,253834,20,8955470,"Exact-Rx, Inc.",Released
4,Dec 27 2022 12:13PM,253834,20,8955471,"Exact-Rx, Inc.",Released
5,Dec 27 2022 12:09PM,253832,10,0086270689,ISDIN Corporation,Released
6,Dec 27 2022 12:09PM,253832,10,0086270690,ISDIN Corporation,Released
7,Dec 27 2022 12:09PM,253832,10,0086270697,ISDIN Corporation,Released
8,Dec 27 2022 12:09PM,253832,10,0086270721,ISDIN Corporation,Released
9,Dec 27 2022 12:09PM,253832,10,0086270722,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,253827,Released,1
35,253828,Released,6
36,253829,Released,3
37,253832,Released,22
38,253834,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253827,NaN,1.0
253828,NaN,6.0
253829,NaN,3.0
253832,NaN,22.0
253834,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253783,0.0,1.0
253784,28.0,1.0
253788,0.0,1.0
253789,0.0,1.0
253790,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253783,0,1
253784,28,1
253788,0,1
253789,0,1
253790,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253783,0,1
1,253784,28,1
2,253788,0,1
3,253789,0,1
4,253790,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253783,,1
1,253784,28,1
2,253788,,1
3,253789,,1
4,253790,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 12:13PM,253834,20,"Exact-Rx, Inc."
5,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation
27,Dec 27 2022 12:02PM,253829,10,ISDIN Corporation
30,Dec 27 2022 11:37AM,253828,10,MedStone Pharma LLC
36,Dec 27 2022 11:35AM,253827,10,"Nextsource Biotechnology, LLC"
37,Dec 27 2022 11:30AM,253826,10,"ClearSpec, LLC"
38,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC"
39,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd"
56,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc."
61,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 12:13PM,253834,20,"Exact-Rx, Inc.",,5
1,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,,22
2,Dec 27 2022 12:02PM,253829,10,ISDIN Corporation,,3
3,Dec 27 2022 11:37AM,253828,10,MedStone Pharma LLC,,6
4,Dec 27 2022 11:35AM,253827,10,"Nextsource Biotechnology, LLC",,1
5,Dec 27 2022 11:30AM,253826,10,"ClearSpec, LLC",,1
6,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",,1
7,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",1,16
8,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",4,1
9,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 12:13PM,253834,20,"Exact-Rx, Inc.",5,
1,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,22,
2,Dec 27 2022 12:02PM,253829,10,ISDIN Corporation,3,
3,Dec 27 2022 11:37AM,253828,10,MedStone Pharma LLC,6,
4,Dec 27 2022 11:35AM,253827,10,"Nextsource Biotechnology, LLC",1,
5,Dec 27 2022 11:30AM,253826,10,"ClearSpec, LLC",1,
6,Dec 27 2022 11:15AM,253825,19,"Emersa Waterbox, LLC",1,
7,Dec 27 2022 11:00AM,253824,19,"AdvaGen Pharma, Ltd",16,1
8,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",1,4
9,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 12:13PM,253834,20,"Exact-Rx, Inc.",5,
1,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,22,
2,Dec 27 2022 12:02PM,253829,10,ISDIN Corporation,3,
3,Dec 27 2022 11:37AM,253828,10,MedStone Pharma LLC,6,
4,Dec 27 2022 11:35AM,253827,10,"Nextsource Biotechnology, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 12:13PM,253834,20,"Exact-Rx, Inc.",5.0,NaN
1,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,22.0,NaN
2,Dec 27 2022 12:02PM,253829,10,ISDIN Corporation,3.0,NaN
3,Dec 27 2022 11:37AM,253828,10,MedStone Pharma LLC,6.0,NaN
4,Dec 27 2022 11:35AM,253827,10,"Nextsource Biotechnology, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 12:13PM,253834,20,"Exact-Rx, Inc.",5.0,0.0
1,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,22.0,0.0
2,Dec 27 2022 12:02PM,253829,10,ISDIN Corporation,3.0,0.0
3,Dec 27 2022 11:37AM,253828,10,MedStone Pharma LLC,6.0,0.0
4,Dec 27 2022 11:35AM,253827,10,"Nextsource Biotechnology, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6599046,450.0,1.0
12,253797,0.0,1.0
15,253823,1.0,4.0
16,253806,1.0,0.0
19,1015237,19.0,1.0
20,507618,6.0,28.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6599046,450.0,1.0
1,12,253797,0.0,1.0
2,15,253823,1.0,4.0
3,16,253806,1.0,0.0
4,19,1015237,19.0,1.0
5,20,507618,6.0,28.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,450.0,1.0
1,12,0.0,1.0
2,15,1.0,4.0
3,16,1.0,0.0
4,19,19.0,1.0
5,20,6.0,28.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,450.0
1,12,Released,0.0
2,15,Released,1.0
3,16,Released,1.0
4,19,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,1.0,1.0,4.0,0.0,1.0,28.0
Released,450.0,0.0,1.0,1.0,19.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,1.0,1.0,4.0,0.0,1.0,28.0
1,Released,450.0,0.0,1.0,1.0,19.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,1.0,1.0,4.0,0.0,1.0,28.0
1,Released,450.0,0.0,1.0,1.0,19.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()